In [1]:
import git
g = git.cmd.Git('/root/Projects/Assignment-Projects')
g.pull()

'Already up-to-date.'

In [12]:
from Sparker.SparkLogProcesser.SparkLogFileHandler import setSparkContext
from pyspark import SparkContext
sc = SparkContext() 
setSparkContext(sc)

In [13]:
from Sparker.Logic.LogicTests import *
day2Logs = readParsedLogsFromHDFS(entireDayParsedLogsFolder2)

hdfs://osldevptst01.host.gittigidiyor.net:8020/user/root/Parsed/2016-12-26_parsed has been read by 2017-05-24 17:23:18.697968


In [14]:
from MainSrc.PythonVersionHandler import *
from paths import *

In [15]:
from Sparker.Logic.LogicTests import *
from Sparker.Logic.Trainer import *
#trainPairWiseData(data) 

In [16]:
def printSeparater(c = 3):
    for n in range(c):
        print_('#' * 88)

In [17]:
from sklearn import svm

table = []
def evaluateModelOnData(model, data, dataName = 'Data', modelName = 'Model'):
    labelsAndPreds = data.map(lambda p: (p.label, model.predict(p.features)))
    truePredictionCount = labelsAndPreds.filter(lambda vp: vp[0] == vp[1]).count()
    instanceCount = data.count()
    accuracy = 100 * truePredictionCount / float(instanceCount)
    print_('\n'+modelName, 'has been evaluated on', dataName, 'by', nowStr())
    print_('The result accuracy is %' + '%.3f\n' % (accuracy))
    table[-1].append(accuracy/100.0)
    if dataName == 'testData':
        table[-1].extend(model.weights)
    return labelsAndPreds

def trainPairWiseData(data, dataName = 'Data', modelName = 'Model', evaluate = True):
    print_('Training with SVMWithSGD...')
    model = SVMWithSGD.train(data, iterations=100)
    print_('\n', modelName, 'has been trained on', dataName, 'by', nowStr())##
    print_('The learned weights:\n' + str(model.weights).replace(',', ', ') + '\n')
    if evaluate:
        evaluateModelOnData(model, data, dataName, modelName)
    return model

def runTrainingExperiment(trainData, testData, modelName = 'Model', save = True, outputFolder = Day1_iPhone_6_DataFolder):
    model = trainPairWiseData(trainData, 'trainData', modelName)
    if save:
        modelPath = joinPath(outputFolder, modelName)
        try:
            model.save(sc_(), modelPath)
        except Py4JJavaError:
            pass
        print_(modelPath, 'has been saved successfully by', nowStr())
    return evaluateModelOnData(model, testData, 'testData', modelName)

In [18]:
def evaluateModelOnDataSK(model, data, dataName = 'Data', modelName = 'Model'):
    labelsAndPreds = data.map(lambda p: (p.label, model.predict(p.features)))
    truePredictionCount = labelsAndPreds.filter(lambda vp: vp[0] == vp[1]).count()
    instanceCount = data.count()
    accuracy = 100 * truePredictionCount / float(instanceCount)
    print_('\n'+modelName, 'has been evaluated on', dataName, 'by', nowStr())
    print_('The result accuracy is %' + '%.3f\n' % (accuracy))
    return labelsAndPreds

def trainPairWiseDataSK(data, dataName = 'Data', modelName = 'Model', evaluate = True):
    model = svm.SVC()#kernel =  'poly', degree = 3) #[:size][:size]
    size = 5000
    X =  data.map(lambda p: p.features).collect()
    Y =  data.map(lambda p: p.label).collect()
    print_('Training with SVC...')
    model.fit(X, Y)
    print_('\n', modelName, 'has been trained on', dataName, 'by', nowStr())##
   # print_('The learned weights:\n' + str(model.coef_).replace(',', ', ') + '\n')
    if evaluate:
        evaluateModelOnDataSK(model, data, dataName, modelName)
    return model

def runTrainingExperimentSK(trainData, testData, modelName = 'Model', save = True, outputFolder = Day1_iPhone_6_DataFolder):
    model = trainPairWiseDataSK(trainData, 'trainData', modelName)
    if save:
        modelPath = joinPath(outputFolder, modelName)
        try:
            model.save(sc_(), modelPath)
        except Py4JJavaError:
            pass
        print_(modelPath, 'has been saved successfully by', nowStr())
    return evaluateModelOnDataSK(model, testData, 'testData', modelName)

In [25]:
from DeepLearningToRank.DeepDataHandler import readTrainDataFromPickle
#from DeepLearningToRank.DeepTrainer import *

def featureFilter(trainData, oneHot = False):
    if oneHot:
        return trainData.map(lambda x: LabeledPoint(x.label, list(x.features[1:4]) + list(x.features[5:8]) + list(x.features[12:-2])))
    else:
        return trainData.map(lambda x: LabeledPoint(x.label, list(x.features[1:4]) + list(x.features[5:6]) + list(x.features[7:-2])))

def runExperiment(trainData, modelName = 'Model'):
    printSeparater(1)
    tot = trainData.count()
    testData, trainData = trainData.randomSplit(weights=[0.3, 0.7])
    table[-1].extend([tot, trainData.count(), testData.count()])
    #testData, trainData = trainData[:int(30*len(trainData)/100)], trainData[int(30*len(trainData)/100):]SKSK
    runTrainingExperiment(trainData, testData, modelName = modelName, save = False)
    print_('The experiment for', modelName, 'has finished by', nowStr())
    
def trainPairWiseDataTestKeyword2(keyword, location = 'HDFS', ext = ''):
    printSeparater(2)
    keyword = keyword.replace(' ', '_')
    inputName = 'all_day'
    if location == 'HDFS':
        outputFolder = joinPath(HDFSDataFolder, 'Day1_' + keyword + '_Data')
        #outputFolder = joinPath(joinPath(textTrainDataFolder, 'HDFS'), 'Day1_' + keyword + '_Data')
        trainDataFile = joinPath(outputFolder, inputName + '_' + keyword + '_TrainData' + ext)#_Updated')
        trainData = readTrainDataFromHDFS(trainDataFile)
    elif location == 'oneHot':
        outputFolder = joinPath(joinPath(textTrainDataFolder, 'HDFS'), 'Day1_' + keyword + '_Data')
        outputFolder = joinPath(joinPath(textTrainDataFolder, 'HDFS'), 'Day1_' + keyword + '_Data')
        trainDataFile = joinPath(outputFolder, inputName + '_' + keyword + '_OneHot_TrainData.txt')
        trainData = readTrainDataFromPickle(trainDataFile)
        trainData = sc_().parallelize(trainData)
        trainData = trainData.map(lambda x: LabeledPoint(x[0], x[1]))
    else:
        trainDataFile = joinPath(textTrainDataFolder, inputName + '_' + keyword + '_TrainData.txt')
        trainData = readTrainDataFromPickle(trainDataFile)
        trainData = sc_().parallelize(trainData)
        trainData = trainData.map(lambda x: LabeledPoint(x[0], x[1]))
    print_(trainData.first())
    print_('Data from', location)
    preprocessing = scaleTrainData# normalizeTrainData#
    trainData = preprocessing(trainData)
    table.append([keyword])
    runExperiment(trainData, modelName = 'ExtendedModel')
    trainData = featureFilter(trainData, oneHot = True)#)
    table.append([keyword])
    runExperiment(trainData, modelName = 'FilteredModel')
    printSeparater(2)

In [26]:
#trainPairWiseDataTestKeyword2('iphone 6', 'oneHot')
#trainPairWiseDataTestKeyword2('tv_unitesi', 'HDFS')
#trainPairWiseDataTestKeyword2('bosch', 'HDFS')
#import pickle def savePairWisetrainData(keyword): printSeparater(2) keyword = keyword.replace(' ', '_') inputName = 'allday' outputFolder = joinPath(HDFSDataFolder, 'Day1' + keyword + '_Data') dataTypes = ['_TrainData', '_labeledPairs', '_journeyproducts'] for typ in dataTypes: fileName = inputName + '' + keyword + typ trainDataFile = joinPath(outputFolder, fileName) trainData = sc().textFile(trainDataFile) f = joinPath('/root/Projects/TrainData6', fileName) trainData = trainData.collect() fp = open(f, "wb") #Pickling pickle.dump(trainData, fp) print(f, 'has been saved successfully by', nowStr())

In [27]:
from py4j.protocol import Py4JJavaError
def trainPairWiseDataTest():
    keywords = ['iphone 6', 'nike air max', 'tv unitesi', 'camasir makinesi', 'bosch', 'jant', 'kot ceket']
    for keyword in keywords:
        try:
            trainPairWiseDataTestKeyword2(keyword, ext = '_OneHot')#)
        except Py4JJavaError:
            continue
    for ta in table: print_(str(ta).replace('[', '').replace(']', ''))
        #savePairWisetrainData(keyword)readTrainDataFromHDFS[:5]

In [28]:
def main():
    printSeparater()
    print_('%s:' % nowStr(), 'Running on osldevptst02:Jupyter...')

    trainPairWiseDataTest()

    print_('%s:' % nowStr(), 'DONE')
    printSeparater()


In [29]:
table = []
main()

########################################################################################
########################################################################################
########################################################################################
2017-05-24 21:36:55.465247: Running on osldevptst02:Jupyter...
########################################################################################
########################################################################################
hdfs://osldevptst01.host.gittigidiyor.net:8020/user/root/data/Day1_iphone_6_Data/all_day_iphone_6_TrainData_OneHot has been read successfully by 2017-05-24 21:36:55.514496
(1.0,[0.0,15.0,100.0,-4001.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,75.3,-780.0])
Data from HDFS
835692 instances have been scaled by 2017-05-24 21:37:15.608453
########################################################################################
Training with SVMWithSGD.

In [30]:
for ta in table: print_(str(ta).replace('[', '').replace(']', ''))

'iphone_6', 835692, 584800, 250892, 0.8996203830369357, 0.8997018637501395, -0.015224330758194677, 0.67968483772744759, 0.038540953814833599, 0.017323045083324884, 0.036512527138668038, 0.00066496998144504715, 0.10113291869691483, -0.10174993156602714, -3.601376045046331e-05, 1.7992772214740062e-05, -0.44201501428952888, 0.44201865258485984, -0.00013305985844372129, -0.00044781867667795987, 0.11167188354033879, -0.11116692804507272, 0.00015490520918561712, 0.0016152953706381208, 0.20759796077427833, -0.20650910435065087, -0.00084152430856879873, -0.00049757456134069558, 0.32135799793516079, -0.31966196007330566, -0.0012277697944509127, 1.157891861487091, -0.21088362067153857
'iphone_6', 835692, 584636, 251056, 0.6971842308718587, 0.6976690459499076, 0.58798285564427044, -0.10265084775657482, -0.45789405692724888, 0.0020868548370568288, 0.070292844555347275, -0.069096293063079656, -0.00047966664590373893, 4.4911323974705276e-05, 0.045544044747374106, -0.045503195333106081, -0.0001896435